In [88]:
import pickle
import pandas as pd
import numpy as np
from load_data_old import *
from beam_search_module import *
import ast

In [89]:
stock_df = pd.read_csv('data/stock_data_all_features.csv')
stock_df['growth_target'] = stock_df['growth_target'].apply(ast.literal_eval)
stock_df_all = stock_df.copy()
stock_df = stock_df_all.drop(['target', 'dates', 'symbol'], axis=1)
stock_df_all


,symbol,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,...,growth_best_day_of_week,growth_best_month,growth_best_quarter,growth_best_day_of_month,growth_best_week_of_month,growth_biggest_continuous_increase_perc,growth_biggest_continuous_decrease_perc,growth_biggest_continuous_increase_perc_start,growth_biggest_continuous_decrease_perc_start,growth_target
0,AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,...,3,8,3,25,34,16.80891,-17.72197,93.0,287.0,"[0.0, 1.02, 0.1, -1.31, 2.49, -0.47, -0.47, 3...."
1,LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,...,1,1,1,31,52,18.51535,-8.00000,64.0,69.0,"[0.0, -2.03, 0.0, 2.07, -2.03, 1.72, 1.69, -2...."
2,ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,...,2,9,3,28,37,11.23512,-6.57895,44.0,22.0,"[0.0, -3.33, -0.69, 4.17, -0.67, -0.67, -0.68,..."
3,FPO.F,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,...,0,1,4,3,40,25.00001,-21.52061,76.0,108.0,"[0.0, -0.13, -2.28, -1.17, -6.44, -1.54, -1.57..."
4,M07.F,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,...,4,4,2,14,15,9.97164,-10.17502,21.0,260.0,"[0.0, -0.67, -0.68, 1.36, 0.0, -0.67, -0.68, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7982,INRN.SW,Switzerland,Specialty Industrial Machinery,CHF,Europe/Zurich,EBS,Industrials,1124.0,15.530,1.979479e+09,...,4,4,2,21,13,17.59738,-12.93413,59.0,259.0,"[0.0, -3.21, 0.83, -0.41, 0.62, -0.72, -0.72, ..."
7983,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,...,4,9,3,9,39,13.67696,-9.63956,290.0,64.0,"[0.0, 0.48, 0.0, -4.75, 0.0, 0.0, 0.0, 2.97, 0..."
7984,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,...,6,9,3,23,39,9.76675,-8.57953,197.0,48.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.51, 0.0..."
7985,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,...,4,9,3,25,39,14.62789,-10.78806,65.0,137.0,"[0.0, 0.0, 0.0, 0.0, 2.95, 0.0, 0.0, 0.0, 0.0,..."


In [90]:
column_names = list(stock_df.columns)
beam_width = 20
beam_depth = 3
nr_bins = 8
nr_saved = 50
subgroup_size = len(stock_df) * 0.05
target = 'growth_target'
window_size = 20
target_ind = column_names.index(target)
stock_df['growth_target'] = stock_df['growth_target'].apply(lambda x: x[:len(x) - (len(x) % window_size)])
stock_df_all['growth_target'] = stock_df_all['growth_target'].apply(lambda x: x[:len(x) - (len(x) % window_size)])
print('length growth_target: ',len(stock_df.loc[0, 'growth_target']))

length growth_target:  300


In [91]:
data = stock_df.values.tolist()
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.median(all_time_series, axis=0)
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = categorize_columns_in_order(stock_df, att_columns)
min_quality_improv = 0

In [5]:
results = beam_search_with_constraint(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                      subgroup_size, target, types, window_size, min_quality_improv)


In [6]:
for r in results:
    print(r)
    print('\n')

(7.274882219354637, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001A5D43F9CF0>), ('marketCap', 1977535730.11456, <function gt at 0x000001A5D43F9CF0>)), 491)


(6.803150219406607, (('fullTimeEmployees', 7686.0, <function gt at 0x000001A5D43F9CF0>), ('country', 'Japan', <function eq at 0x000001A5D43F9E10>)), 616)


(6.650918653291399, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('marketCap', 12953903104.0, <function leeq at 0x000001A5D43F9D80>), ('fullTimeEmployees', 1817.0, <function gt at 0x000001A5D43F9CF0>)), 575)


(6.205676446380705, (('country', 'Japan', <function eq at 0x000001A5D43F9E10>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001A5D43F9CF0>)), 656)


(5.8773758485413, (('fullTimeEmployees', 46000.0, <function gt at 0x000001A5D43F9CF0>), ('enterpriseToEbitda', 9.382, <function leeq at 0x000001A5D43F9D80>), ('enterpriseToEbitda', 8.486, <function leeq at 0x000001A5D43F9D80>)), 535)



In [7]:
descriptors = results[0][1]
df = filter_df_on_descriptors(stock_df_index, descriptors)
df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
13,Japan,Auto Manufacturers,EUR,Europe/Berlin,FRA,Consumer Cyclical,0.0,1.347,8.373396e+09,57.726,51249.0,"[0.0, -0.3, 0.13, -2.27, -7.6, 6.83, -5.39, -1..."
79,Japan,Auto Parts,EUR,Europe/Berlin,FRA,Consumer Cyclical,0.0,0.328,2.590374e+10,25.747,135636.0,"[0.0, 0.85, 0.8, -4.17, -5.86, 0.46, -1.03, -3..."
84,Japan,Marine Shipping,EUR,Europe/Berlin,FRA,Industrials,46.0,1.864,3.703204e+09,42.062,35165.0,"[0.0, 3.86, 1.9, -13.63, -12.04, 11.3, 0.12, -..."
148,Japan,Electrical Equipment & Parts,EUR,Europe/Berlin,FRA,Industrials,2.0,0.616,5.795071e+09,24.438,26757.0,"[0.0, 5.96, 5.33, -5.67, -8.94, 3.39, 3.97, 7...."
159,Japan,Consumer Electronics,EUR,Europe/Berlin,FRA,Technology,2190.0,0.961,2.100761e+10,43.900,241423.0,"[0.0, -2.49, -3.51, 7.56, -10.28, 5.86, -5.7, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9645,Japan,Auto Manufacturers,ARS,America/Argentina/Buenos_Aires,BUE,Consumer Cyclical,145.0,8.486,3.974523e+12,102.678,376971.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 97.3,..."
9774,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
